In [4]:
import streamlit as st
from utils import extract_recipe_details, extract_timestamps
from youtube_scrapper import get_eng_recipe
from gemini import get_gemini_response, create_prompt_for_recipe, create_prompt_for_timestamps

recipe_to_cook = "shahi paneer"
eng_recipes = get_eng_recipe(f"{recipe_to_cook} recipe")
index_of_recipe_chosen = 0
selected_video = eng_recipes[index_of_recipe_chosen]
video_id = selected_video['id']
base_youtube_video_url = f"https://www.youtube.com/embed/{video_id}"

In [1]:
from youtube_search import YoutubeSearch
from youtube_transcript_api import YouTubeTranscriptApi

def get_eng_recipe(recipe_name, min_duration = 5):
    '''this function uses transcript api to find recipes with eng subtitles and list them,
    and returns a list of dictionaries with video details & transcript'''
    eng_recipe = []
    results = YoutubeSearch(recipe_name, max_results=100).to_dict()
    for result in results:
        # checking if the duration > min duration
        if float(result['duration'].split(':')[0]) >= min_duration: 
            try:
                transcript_list = YouTubeTranscriptApi.list_transcripts(result['id'])
                for transcript in transcript_list:
                    # check if the lang is english
                    if 'english' in transcript.language.lower():
                        result['transcript'] = YouTubeTranscriptApi.get_transcript(result['id'], ['en', 'en-GB', 'en-US'])
                        eng_recipe.append(result)
                    if len(eng_recipe) == 10:
                        break
            except:
                None
    return eng_recipe

In [3]:
answer = get_eng_recipe('shahi paneer')

In [4]:
answer[0].keys()

dict_keys(['id', 'thumbnails', 'title', 'long_desc', 'channel', 'duration', 'views', 'publish_time', 'url_suffix', 'transcript'])

In [6]:
def fetch_recipe_details(selected_video, recipe_to_cook, gemini_api_key):
    role, recipe_prompt = create_prompt_for_recipe(v, recipe_to_cook)
    recipe_output = get_gemini_response(role, recipe_prompt, gemini_api_key)
    steps, ingredients, servings, utensils = extract_recipe_details(recipe_output)
    return steps, ingredients, servings, utensils

def fetch_recipe_steps_timestamps(selected_video, steps, recipe_to_cook, gemini_api_key):
    role, timestamp_prompt = create_prompt_for_timestamps(selected_video['transcript'], steps, recipe_to_cook)
    timestamp_output = get_gemini_response(role, timestamp_prompt, gemini_api_key)
    return extract_timestamps(timestamp_output)

steps, ingredients, servings, utensils = fetch_recipe_details(selected_video, recipe_to_cook)
timestamps = fetch_recipe_steps_timestamps(selected_video, steps, recipe_to_cook)

In [8]:
def create_prompt_for_question_answering(transcript, recipe_to_cook, question):
    model_role = "As a chef with expertise in various recipes, your task is to provide detailed answers to questions based on a specific recipe's transcript. Use your culinary knowledge and the given transcript to offer insightful and accurate responses."

    prompt = f"""Transcript of the {recipe_to_cook} Recipe: \n\n{transcript}\n\nGiven the detailed transcript above for the {recipe_to_cook} recipe, your expertise is needed to answer this culinary question:\n\nQuestion: {question}\n\nAs a chef, your Answer:"""

    return model_role, prompt

question = "what are the changes that i need to make to the recipe to make it vegan?"
qa_role, qa_prompt = create_prompt_for_question_answering(selected_video['transcript'], recipe_to_cook, question)
answer = get_gemini_response(qa_role, qa_prompt)
answer

'To make the shahi paneer recipe vegan, the following changes need to be made:\n\n1. **Replace the butter:** Instead of using butter to make the curry, use a vegan butter alternative such as plant-based margarine or vegan ghee.\n\n2. **Substitute the cream:** To achieve the creamy texture in the curry without using dairy cream, you can use coconut cream, cashew cream, or almond milk.\n\n3. **Omit the paneer:** As paneer is a type of cheese, it is not suitable for a vegan diet. You can replace the paneer with tofu, tempeh, or a plant-based meat alternative.\n\n4. **Double-check the other ingredients:** Make sure that all the other ingredients you use, such as spices, herbs, and seasonings, are vegan-friendly.\n\nBy making these changes, you can create a delicious and satisfying vegan version of shahi paneer that is just as flavorful and creamy as the traditional recipe.'

In [11]:
print(answer)

To make the shahi paneer recipe vegan, the following changes need to be made:

1. **Replace the butter:** Instead of using butter to make the curry, use a vegan butter alternative such as plant-based margarine or vegan ghee.

2. **Substitute the cream:** To achieve the creamy texture in the curry without using dairy cream, you can use coconut cream, cashew cream, or almond milk.

3. **Omit the paneer:** As paneer is a type of cheese, it is not suitable for a vegan diet. You can replace the paneer with tofu, tempeh, or a plant-based meat alternative.

4. **Double-check the other ingredients:** Make sure that all the other ingredients you use, such as spices, herbs, and seasonings, are vegan-friendly.

By making these changes, you can create a delicious and satisfying vegan version of shahi paneer that is just as flavorful and creamy as the traditional recipe.
